In [8]:
!pip install -U pip
!pip install u-msgpack-python
!pip install pydrake
!pip install -U numpy
!pip install underactuated
!pip install manipulation
!pip install pyngrok
!git clone https://github.com/RussTedrake/meshcat-python --branch colab --recursive
import sys
sys.path.append('/content/meshcat-python/src')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'meshcat-python' already exists and is not an empty directory.


# Initalize simulation



In [9]:
import numpy as np
import pydot
import mpld3
import subprocess
import re
import time
from pyngrok import ngrok, conf
import google.colab.output as output
from meshcat.jupyter import JupyterVisualizer
from underactuated import ConfigureParser, running_as_notebook
from IPython.display import HTML, SVG, display, Javascript
from ipykernel import comm
from underactuated.meshcat_utils import MeshcatSliders
from underactuated.scenarios import AddFloatingRpyJoint
from pydrake.all import ( 
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    GenerateHtml,
    InverseDynamicsController,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    Parser,
    Simulator,
    StartMeshcat,
    DrakeLcmInterface,    
    Propeller,
    PropellerInfo,
    RigidTransform,
    SceneGraph
)
#mpld3.enable_notebook()

# Visualizer things

In [10]:
"""
Initalize meshcat local server
"""
mesh_cat = StartMeshcat()
port_id = mesh_cat.port()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [11]:
"""
Tunnel local port to ngrok https server url (publically avaliable url)
"""
token = '2HZ1KxmMHUPsgDFidy4Etnb0Vlp_3xFiqS1JAD7PpV58hwrK1'
config = conf.PyngrokConfig(auth_token=token, start_new_session=True)
public_url = ngrok.connect(mesh_cat.web_url(), "http", pyngrok_config=config)
print(public_url)

NgrokTunnel: "https://5e21-35-230-69-226.ngrok-free.app" -> "http://localhost:7001"


In [13]:

#mesh_cat.DeleteAddedControls() #? resets meshcat
builder = DiagramBuilder()

#plant = builder.AddSystem(MultibodyPlant(0.0))
#scene_graph = builder.AddSystem(SceneGraph())
# Adds both MultibodyPlant and the SceneGraph, and wires them together.
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
# Note that we parse into both the plant and the scene_graph here.
parser = Parser(plant)
ConfigureParser(parser)
(model_instance,) = parser.AddModelsFromUrl(
    "package://drake/examples/quadrotor/quadrotor.urdf"
)
#(model_instance,)=Parser(plant, scene_graph).AddModelsFromUrl(
#        "package://drake/examples/quadrotor/quadrotor.urdf"
#)

AddFloatingRpyJoint(
    plant,
    plant.GetFrameByName("base_link"),
    model_instance,
    use_ball_rpy=False,
)


#plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base_link"))
plant.Finalize()

body_index = plant.GetBodyByName("base_link").index()
# Default parameters from quadrotor_plant.cc:
L = 0.15  # Length of the arms (m).
kF = 1.0  # Force input constant.
kM = 0.0245  # Moment input constant.
# Note: Rotors 0 and 2 rotate one way and rotors 1 and 3 rotate the other.
prop_info = [
    PropellerInfo(body_index, RigidTransform([L, 0, 0]), kF, kM),
    PropellerInfo(body_index, RigidTransform([0, L, 0]), kF, -kM),
    PropellerInfo(body_index, RigidTransform([-L, 0, 0]), kF, kM),
    PropellerInfo(body_index, RigidTransform([0, -L, 0]), kF, -kM),
]

propellers = builder.AddSystem(Propeller(prop_info))

builder.Connect(
    propellers.get_output_port(),
    plant.get_applied_spatial_force_input_port(),
)

builder.Connect(
    plant.get_body_poses_output_port(),
    propellers.get_body_poses_input_port(),
)

builder.ExportInput(propellers.get_command_input_port(), "u")


"""
Parser(plant, scene_graph).AddModelsFromUrl(
    "package://drake/manipulation/models/iiwa_description/sdf/iiwa14_no_collision.sdf"
)
"""
#attaches multibody plant to world frame (would fall through environment ground)
#plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_link_0"))
#plant.Finalize()



# Adds the MeshcatVisualizer and wires it to the SceneGraph.
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, mesh_cat)


mbp_plant = builder.Build()
context = mbp_plant.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)
#plant.get_actuation_input_port(0).FixValue(plant_context, np.array([0,0,0,0], dtype=np.float64))
#diagram = builder.Build()
#context = diagram.CreateDefaultContext()
#plant_context = plant.GetMyMutableContextFromRoot(context)
#plant.get_actuation_input_port().FixValue(plant_context, np.zeros(4))
#diagram = builder.Build()



#SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg())

mbp_plant.set_name("diagram")

display(HTML(
    '<script src="https://unpkg.com/gojs/release/go.js"></script>'
    + GenerateHtml(mbp_plant)
))


simulator = Simulator(mbp_plant)
context = simulator.get_mutable_context()
plant_context = plant.GetMyMutableContextFromRoot(context)

simulator.set_target_realtime_rate(1.0)
print("Use the slider in the MeshCat controls to apply elbow torque.")
print("Press 'Stop Simulation' in MeshCat to continue.")
mesh_cat.AddButton("Stop Simulation")

rng = np.random.default_rng(seed=1037)
while mesh_cat.GetButtonClicks("Stop Simulation") < 1:
    #u = rng.random((4,))
    u = np.array([2, 2, 2, 2])
    #mbp_x = rng.random((12,))
    mbp_plant.get_input_port().FixValue(context, u)
    #plant_context.SetContinuousState(mbp_x)
    simulator.AdvanceTo(simulator.get_context().get_time() + 1.0)
    time.sleep(1.0)
#context = diagram.CreateDefaultContext()
#diagram.ForcedPublish(context)

simulator.AdvanceTo(5.0)



Use the slider in the MeshCat controls to apply elbow torque.
Press 'Stop Simulation' in MeshCat to continue.
